In [71]:
from pathlib import Path
import swat
import pandas as pd
import numpy as np
import sklearn.datasets
import lightgbm as lgb
from itertools import combinations
from sklearn.metrics import roc_auc_score

fetch_opts = dict(to=1000000, maxrows=1000000)
dpp_opts = dict(distinctCountLimit=1000000, misraGries=False)
dsp_opts = dict(distinctCountLimit=1000000, misraGries=False)

# Set the table Index (the current train/test table to work on)

In [72]:

openml_datasets_dict = dict()
 

tbl_index = 4

tbl_keys = list(openml_datasets_dict.keys())

if tbl_index == 0:
    openml_datasets_dict[1111] = "KDDCup09_appetency"
elif tbl_index == 1:
    openml_datasets_dict[179] = "adult"
elif tbl_index == 2:
    openml_datasets_dict[23512] = "higgs"
elif tbl_index == 3:
    openml_datasets_dict[3] = "kr_vs_kp"
elif tbl_index == 4:
    openml_datasets_dict[41138] = "APSFailure"
elif tbl_index == 5:
    openml_datasets_dict[41146] = "sylvine"
elif tbl_index == 6:
    openml_datasets_dict[1461] = "bank_marketing"
elif tbl_index == 7:
    openml_datasets_dict[1486] = "nomao"
elif tbl_index == 8:
    openml_datasets_dict[41142] = "christine"
elif tbl_index == 9:
    openml_datasets_dict[41150] = "MiniBooNE"
    


In [73]:
data_path = "C:\\users\\bigebr\\Desktop\\openml_automl_data"

"""

# first time, uncomment and download dataset with this 


for index in openml_datasets_dict:
    print("Fetching: {0}".format(openml_datasets_dict[index]))
    data_bunch = sklearn.datasets.fetch_openml(data_id=index, data_home=data_path, 
                                  target_column='default-target', cache=False, 
                                  return_X_y=False)
    y = data_bunch.target
    X = data_bunch.data
    df = pd.DataFrame(X, columns=['x'+str(i) for i in range(1, X.shape[1]+1)])
    df['y'] = y
    p = Path(data_path)
    p = p / str(openml_datasets_dict[index] + ".csv")
    df.to_csv(p, index=False)
    
"""

In [74]:
server = "rdcgrd133.unx.sas.com"
port = 16271

s = swat.CAS(server, port)

In [75]:
s.loadactionset(actionset='dataPreprocess')
s.loadactionset(actionset='dataSciencePilot')
s.loadactionset(actionset='astore')
s.loadactionset(actionset='simple')
s.loadactionset(actionset='sampling')

NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'dataPreprocess'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.000405 seconds
NOTE:       cpu time                0.000000 seconds (0.00%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  208.78K (0.00%)
NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'dataSciencePilot'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.047516 seconds
NOTE:       cpu time                0.003999 seconds (8.42%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  225.91K (0.00%)
NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'astore'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time  

[actionset]

 'sampling'

+ Elapsed: 0.00466s, user: 0.002s, sys: 0.002s, mem: 0.213mb

In [76]:
for index in openml_datasets_dict:
    p = Path(data_path)
    p = p / str(openml_datasets_dict[index] + ".csv")
    print("Uploading: {0}".format(openml_datasets_dict[index]))
    s.upload(str(p), casout=dict(name=openml_datasets_dict[index], replace=True))

Uploading: APSFailure
NOTE: Executing action 'table.loadTable'.
NOTE: Cloud Analytic Services made the uploaded file available as table APSFAILURE in caslib CASUSER(bigebr).
NOTE: Action 'table.loadTable' used (Total process time):
NOTE:       real time               0.836612 seconds
NOTE:       cpu time                5.579152 seconds (666.87%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  158.95M (0.06%)
NOTE: The table APSFAILURE has been created in caslib CASUSER(bigebr) from binary data uploaded to Cloud Analytic Services.
NOTE: Action 'table.upload' used (Total process time):
NOTE:       real time               2.261812 seconds
NOTE:       cpu time                6.036082 seconds (266.87%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  158.95M (0.06%)


In [77]:
s.tableInfo()

NOTE: Executing action 'table.tableInfo'.
NOTE: Action 'table.tableInfo' used (Total process time):
NOTE:       real time               0.001208 seconds
NOTE:       cpu time                0.000000 seconds (0.00%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  701.00K (0.00%)


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,APSFAILURE,76000,171,0,utf-8,2020-10-10T18:11:06-04:00,2020-10-10T18:11:06-04:00,2020-10-10T18:11:06-04:00,UTF8,1.917987e+09,...,0,0,0,,,0,bigebr,,2020-10-10T18:11:06-04:00,1.917987e+09


In [78]:
# check binary classification sanity
target = 'y'
for index in openml_datasets_dict:
    temp_tbl = openml_datasets_dict[index]
    res = s.dataPreprocess.highCardinality(table=temp_tbl, inputs=target)
    if int(res['HighCardinalityDetails']['CardinalityEstimate'][0]) != 2:
        print(res)
        raise Exception("{0}: not a binary classification".format(temp_tbl))

NOTE: Executing action 'dataPreprocess.highCardinality'.
NOTE: Action 'dataPreprocess.highCardinality' used (Total process time):
NOTE:       real time               0.009325 seconds
NOTE:       cpu time                0.034996 seconds (375.29%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  4.72M (0.00%)


In [79]:
def encode_target(s, tbl, target, event, out_tbl_name='_out1_'):
    # code target as [0,1] with rare/event labelled as 1.
    # this casts multinomial into binary.
    freq_tbl = s.freq(table=tbl, inputs=target)['Frequency']
    freqs = list(freq_tbl['Frequency'])
    numeric_target = False
    if 'NumVar' in list(freq_tbl.columns):
        numeric_target = True
    col_name = 'NumVar' if numeric_target else 'CharVar'
    col_vals = list(freq_tbl[col_name])
    if not numeric_target:
        col_vals = [c.lower() for c in col_vals]
    if event:
        event = event.strip()
        if numeric_target:
            event = int(event)
            event_index = col_vals.index(event)
        else:
            event_index = col_vals.index(event.lower())                
    else:
        # index of the rare element
        event_index = min(enumerate(freqs), key=itemgetter(1))[0] 
    event = str(col_vals[event_index]).strip()
    temp_out_name = out_tbl_name
    encoded_target_name = target.lower()+"_encoded"
    event_formatted = event.lower()
    if not numeric_target:
        event_formatted = "'{0}'".format(event.lower())
    code_str ="""
            data {0} (drop={1});
                set {2};
                if lowcase({3}) = {4} then {5} = 1;
                else {6} = 0;
            run;
        """.format(
            temp_out_name,
            target,
            tbl,
            target,
            event_formatted,
            encoded_target_name,
            encoded_target_name                
        )
    s.dataStep.runCode(code=code_str)
    print(code_str)
    return temp_out_name, encoded_target_name

In [80]:
for index in openml_datasets_dict:
    curr_tbl = openml_datasets_dict[index]
    temp_df = s.freq(table=curr_tbl, inputs=target)['Frequency']
    if "NumVar" in temp_df.columns:
        col_key = "NumVar"
        numeric = True
    else:
        col_key = "FmtVar"
        numeric = False
    levels = list(temp_df[col_key])
    freqs = list(temp_df['Frequency'])
    level_freqs = dict(zip(levels, freqs))
    rare_level = levels[np.argmin(freqs)]
    if numeric:
        event = str(int(rare_level))
    else:
        event = str(rare_level)
    print("rare level: {0}".format(event))

    encode_target(s, curr_tbl, target,  event, out_tbl_name=curr_tbl+"_target_0_1")
for index in openml_datasets_dict:
    s.droptable(table=openml_datasets_dict[index])
    openml_datasets_dict[index] += "_target_0_1" 

NOTE: Executing action 'simple.freq'.
NOTE: Action 'simple.freq' used (Total process time):
NOTE:       real time               0.011227 seconds
NOTE:       cpu time                0.055992 seconds (498.73%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  4.50M (0.00%)
rare level: pos
NOTE: Executing action 'simple.freq'.
NOTE: Action 'simple.freq' used (Total process time):
NOTE:       real time               0.011762 seconds
NOTE:       cpu time                0.061991 seconds (527.04%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  4.50M (0.00%)
NOTE: Executing action 'dataStep.runCode'.
NOTE: Action 'dataStep.runCode' used (Total process time):
NOTE:       real time               0.200075 seconds
NOTE:       cpu time                1.485774 seconds (742.61%)
NOTE:       total nodes             1 (32 cores)
NOTE:       tot

In [81]:
s.tableInfo()

NOTE: Executing action 'table.tableInfo'.
NOTE: Action 'table.tableInfo' used (Total process time):
NOTE:       real time               0.001274 seconds
NOTE:       cpu time                0.001000 seconds (78.49%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  701.81K (0.00%)


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,APSFAILURE_TARGET_0_1,76000,171,0,utf-8,2020-10-10T18:11:22-04:00,2020-10-10T18:11:22-04:00,2020-10-10T18:11:22-04:00,UTF8,1.917987e+09,...,0,0,0,,,0,bigebr,,,NaN


In [82]:
# verify that target's rare is coded as "1"
for index in openml_datasets_dict:
    curr_tbl = openml_datasets_dict[index]
    temp_df = s.freq(table=curr_tbl, inputs='y_encoded')['Frequency']
    print(temp_df)

NOTE: Executing action 'simple.freq'.
NOTE: Action 'simple.freq' used (Total process time):
NOTE:       real time               0.009044 seconds
NOTE:       cpu time                0.060990 seconds (674.37%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  4.50M (0.00%)
Frequency for APSFAILURE_TARGET_0_1

      Column  NumVar        FmtVar  Level  Frequency
0  y_encoded     0.0             0      1    74625.0
1  y_encoded     1.0             1      2     1375.0


In [83]:
openml_datasets_dict

{41138: 'APSFailure_target_0_1'}

In [84]:
target = 'y_encoded'

# Slice train/test :- 80/20 

In [85]:
class TrainTestSlicer:
    
    def __init__(self, s, input_tbl):
        self._session = s
        self._input_tbl = input_tbl
        s.loadactionset(actionset='sampling')
        
    def run(self, test_slice_percent=20, seed=123):
        s = self._session
        s.sampling.srs(table=self._input_tbl, 
                       samppct = test_slice_percent,
                       seed = seed,
                       partInd = True,
                       output=dict(casout=dict(name='temp', replace=True),
                                   copyVars = "ALL"))
        all_vars = s.columnInfo(table=self._input_tbl)['ColumnInfo']['Column']
        all_vars_except_part_ind = list(set([v.lower() for v in all_vars]) - \
                                        set(['_partind_']))
        train_tbl = self._input_tbl + "_train"
        self._train_tbl = train_tbl
        s.partition(table=dict(name='temp', vars = all_vars_except_part_ind,
                            where="_PartInd_=0"), 
                    casout=dict(name=train_tbl, replace=True))
        test_tbl = self._input_tbl + "_test"
        self._test_tbl = test_tbl
        s.partition(table=dict(name='temp', vars = all_vars_except_part_ind,
                            where="_PartInd_=1"), 
                    casout=dict(name=test_tbl, replace=True))
        s.dropTable(table='temp')
        return train_tbl, test_tbl
    
    @property
    def train_tbl(self):
        return self._train_tbl

    @property
    def test_tbl(self):
        return self._test_tbl


In [86]:
for index in openml_datasets_dict:
    tbl = openml_datasets_dict[index]
    slicer = TrainTestSlicer(s, tbl)
    train_tbl, test_tbl = slicer.run(test_slice_percent=20, seed=123)
    print(train_tbl, test_tbl)

NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'sampling'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.000482 seconds
NOTE:       cpu time                0.000000 seconds (0.00%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  209.75K (0.00%)
NOTE: Executing action 'sampling.srs'.
NOTE: Simple Random Sampling is in effect.
NOTE: Using SEED=123 for sampling.
NOTE: Action 'sampling.srs' used (Total process time):
NOTE:       real time               0.168090 seconds
NOTE:       cpu time                1.161824 seconds (691.19%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  107.35M (0.04%)
NOTE: Executing action 'table.columnInfo'.
NOTE: Action 'table.columnInfo' used (Total process time):
NOTE:       real time               0.002308 seconds
NOTE

In [87]:
s.tableInfo()

NOTE: Executing action 'table.tableInfo'.
NOTE: Action 'table.tableInfo' used (Total process time):
NOTE:       real time               0.001446 seconds
NOTE:       cpu time                0.000999 seconds (69.09%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  703.66K (0.00%)


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,APSFAILURE_TARGET_0_1,76000,171,0,utf-8,2020-10-10T18:11:22-04:00,2020-10-10T18:11:22-04:00,2020-10-10T18:11:30-04:00,UTF8,1.917987e+09,...,0,0,0,,,0,bigebr,,,NaN
1,APSFAILURE_TARGET_0_1_TRAIN,60800,171,0,utf-8,2020-10-10T18:11:30-04:00,2020-10-10T18:11:30-04:00,2020-10-10T18:11:30-04:00,UTF8,1.917987e+09,...,0,0,0,,,0,bigebr,,,NaN
2,APSFAILURE_TARGET_0_1_TEST,15200,171,0,utf-8,2020-10-10T18:11:30-04:00,2020-10-10T18:11:30-04:00,2020-10-10T18:11:30-04:00,UTF8,1.917987e+09,...,0,0,0,,,0,bigebr,,,NaN


In [88]:
tbl_base_name = list(openml_datasets_dict.values())[0] 
curr_train_tbl = tbl_base_name + "_TRAIN"
curr_test_tbl = tbl_base_name + "_TEST"
print(curr_train_tbl)
print(curr_test_tbl)

APSFailure_target_0_1_TRAIN
APSFailure_target_0_1_TEST


# Data exploration (to identify nominal/categorical vs interval/continuous vars)

In [89]:
s.dataSciencePilot.exploreData(table=curr_train_tbl, target=target, casout=dict(name='avapt_out', replace=True),
                               **dsp_opts)
curr_avapt_df = s.fetch(table='avapt_out', **fetch_opts)['Fetch']


all_predictor_vars = curr_avapt_df['Variable']
types = curr_avapt_df['VarType']

# numerics is not a good term, really.
nominals = []
intervals = []
for i, v in enumerate(all_predictor_vars):
    # first row is always target
    if i == 0:
        continue
    if "nominal" in types[i]:
        nominals.append(v)
    else:
        intervals.append(v)
# identify target event
temp_df = s.freq(table=curr_train_tbl, inputs=target)['Frequency']
if "NumVar" in temp_df.columns:
    col_key = "NumVar"
    numeric = True
else:
    col_key = "FmtVar"
    numeric = False
levels = list(temp_df[col_key])
freqs = list(temp_df['Frequency'])
level_freqs = dict(zip(levels, freqs))
rare_level = levels[np.argmin(freqs)]
if numeric:
    event = str(int(rare_level))
else:
    event = str(rare_level)
print("rare level: {0}".format(int(event)))

screen_pol = dict(
    missingPercentThreshold = 75,
    constant = True,
    lowCv = True,
    redundant = 1,
    lowInformation = 0.01,
    leakagePercentThreshold = 90,
    missingIndicatorPercent = 50,
    groupRareLevels = True)

s.dataSciencePilot.screenVariables(table=curr_train_tbl, target=target, 
                                   screenPolicy = screen_pol,
                                   casout= dict(name='sweeper_out', replace=True))
curr_sweeper_df = s.fetch(table='sweeper_out', **fetch_opts)['Fetch']

# keep vars that are strictly 'keep'
keep_vars = curr_sweeper_df.query("Recommendation == 'keep'")['Variable']

kept_nominals = list(set(nominals).intersection(set(keep_vars)))
kept_intervals = list(set(intervals).intersection(set(keep_vars)))

s.dataSciencePilot.selectFeatures(table=curr_train_tbl,
                                  target=target,
                                  event=event,
                                  inputs = kept_intervals + kept_nominals,
                                  nominals = [target] + kept_nominals,
                                  selectionPolicy=dict(criterion='su', topK=100),
                                  casout=dict(name='select_out', replace=True))                                  
curr_select_df = s.fetch(table='select_out', **fetch_opts)['Fetch']
ranked_vars = list(curr_select_df['Variable'])
print(ranked_vars)
ranked_intervals = []
ranked_nominals = []
for v in ranked_vars:
    if v in kept_intervals:
        ranked_intervals.append(v)
    else:
        ranked_nominals.append(v)

NOTE: Executing action 'dataSciencePilot.exploreData'.
NOTE: Action 'dataSciencePilot.exploreData' used (Total process time):
NOTE:       real time               4.175738 seconds
NOTE:       cpu time                45.162135 seconds (1081.54%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  247.94M (0.10%)
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               0.003190 seconds
NOTE:       cpu time                0.002999 seconds (94.01%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  1.10M (0.00%)
NOTE: Executing action 'simple.freq'.
NOTE: Action 'simple.freq' used (Total process time):
NOTE:       real time               0.010854 seconds
NOTE:       cpu time                0.064991 seconds (598.77%)
NOTE:       total nodes             1 (32 cores)
NOTE

In [97]:
curr_avapt_df['MissingRated']

0      NaN
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
166    1.0
167    1.0
168    1.0
169    1.0
170    1.0
Name: MissingRated, Length: 171, dtype: float64

In [90]:
# 1. lasso logistic vs lasso (logistic with interactions) vs lightgbm

0      0.000000
1      1.062500
2      4.633224
3      1.134868
4      4.631579
         ...   
166    4.633224
167    1.128289
168    1.128289
169    4.629934
170    0.569079
Name: MissRate, Length: 171, dtype: float64

In [105]:
# first preprocess
high_miss_vars = set(curr_avapt_df.query("MissRate > 1")['Variable'])
low_miss_vars = set(curr_avapt_df.query("MissRate <= 1")['Variable'])
high_miss_intervals = list(set(kept_intervals).intersection(high_miss_vars))
low_miss_intervals = list(set(kept_intervals).intersection(low_miss_vars))
high_miss_nominals = list(set(kept_nominals).intersection(high_miss_vars))
low_miss_nominals = list(set(kept_nominals).intersection(low_miss_vars))

In [110]:
p1 = dict(name='tr1',
          inputs=low_miss_intervals,
          impute=dict(method='median'))
tr1_low_miss_intervals = ['tr1_'+v for v in low_miss_intervals]
p2 = dict(name='tr2',
          inputs=high_miss_intervals,
          impute=dict(method='median'))
tr2_high_miss_intervals = ['tr2_'+v for v in high_miss_intervals]
p2 = dict(name='tr3',
          inputs=high_miss_intervals,
          hash=dict(method='missIndicator'))
tr3_high_miss_intervals = ['tr3_'+v for v in high_miss_intervals]
p3 = dict(name='tr4',
          inputs=low_miss_nominals,
          impute=dict(method='mode'))
tr4_low_miss_nominals = ['tr4_'+v for v in low_miss_nominals]
p4 = dict(name='tr5',
          inputs=high_miss_nominals,
          cattrans = dict(method='label',
                          args=dict(overrides=dict(binMissing=True))))
tr5_high_miss_nominals = ['tr5_'+v for v in high_miss_nominals]
pipelines = [p1, p2, p3, p4]
s.dataPreprocess.transform(table=curr_train_tbl,
                           pipelines = pipelines,
                           savestate=dict(name='_preprocess_astore_', replace=True))

NOTE: Executing action 'dataPreprocess.transform'.
NOTE: 170149 bytes were written to the table "_preprocess_astore_" in the caslib "CASUSER(bigebr)".
NOTE: Action 'dataPreprocess.transform' used (Total process time):
NOTE:       real time               0.464286 seconds
NOTE:       cpu time                5.092226 seconds (1096.79%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  42.33M (0.02%)


[TransInfo]

 Transformation Requests for APSFAILURE_TARGET_0_1_TRAIN
 
   Name  NTransVars                                 DeactivationReason  \
 0  tr1          10                                                      
 1  tr3         118                                                      
 2  tr4           0  No qualifying variables could be found for the...   
 3  tr5          21                                                      
 
   ImputeMethod           Hash  HashNBuckets          CatTransMethod  
 0       Median                          NaN                          
 1               MissIndicator           2.0                          
 2         Mode                          NaN                          
 3                                       NaN  Label (Sparse One-Hot)  

[VarTransInfo]

 Variable Transformation Information for APSFAILURE_TARGET_0_1_TRAIN
 
     Variable Transformation ResultVar      N  NMiss  ImputedValueContinuous
 0         x1            tr1    tr1_x1  60800      0                 30788.0
 1        x10            tr3   tr3_x10  60112    688                     NaN
 2       x100            tr3  tr3_x100  60093    707                     NaN
 3       x101            tr3  tr3_x101  60093    707                     NaN
 4       x102            tr3  tr3_x102  60093    707                     NaN
 ..       ...            ...       ...    ...    ...                     ...
 144       x6            tr5    tr5_x6  58222   2578                     NaN
 145       x8            tr5    tr5_x8  60112    688                     NaN
 146      x92            tr5   tr5_x92  45707  15093                     NaN
 147      x93            tr5   tr5_x93  45707  15093                     NaN
 148      x98            tr5   tr5_x98  51215   9585                     NaN
 
 [149 rows x 6 columns]

[NomVarInfo]

 Nominal Variable Information for APSFAILURE_TARGET_0_1_TRAIN
 
    Variable        N    NMiss  NLevels
 0      x122  60114.0    686.0    777.0
 1      x123  60114.0    686.0     57.0
 2      x128  46758.0  14042.0    747.0
 3      x130  46758.0  14042.0    256.0
 4      x131  46758.0  14042.0    141.0
 5      x135  56685.0   4115.0    401.0
 6      x155  57986.0   2814.0    130.0
 7       x19  60171.0    629.0    895.0
 8       x27  57985.0   2815.0     69.0
 9       x33  60110.0    690.0    463.0
 10      x34  60110.0    690.0    919.0
 11      x35  60110.0    690.0    982.0
 12      x36  60110.0    690.0   1031.0
 13      x42  60110.0    690.0    462.0
 14       x5  58222.0   2578.0    320.0
 15      x52  60110.0    690.0    330.0
 16       x6  58222.0   2578.0    399.0
 17       x8  60112.0    688.0    660.0
 18      x92  45707.0  15093.0    512.0
 19      x93  45707.0  15093.0    651.0
 20      x98  51215.0   9585.0    995.0

[OutputCasTables]

             casLib                 Name  Rows  Columns  \
 0  CASUSER(bigebr)  _preprocess_astore_     1        2   
 
                                             casTable  
 0  CASTable('_preprocess_astore_', caslib='CASUSE...  

+ Elapsed: 0.464s, user: 5s, sys: 0.096s, mem: 42.3mb

In [111]:
preprocessed_curr_train_tbl = 'preprocessed_curr_train_tbl'
s.astore.score(table=curr_train_tbl,
               rstore='_preprocess_astore_',
               copyvars = target,
               casout = dict(name=preprocessed_curr_train_tbl, replace=True))
preprocessed_curr_test_tbl = 'preprocessed_curr_test_tbl'
s.astore.score(table=curr_test_tbl,
               rstore='_preprocess_astore_',
               copyvars = target,
               casout = dict(name=preprocessed_curr_test_tbl, replace=True))


NOTE: Executing action 'astore.score'.
NOTE: Action 'astore.score' used (Total process time):
NOTE:       real time               0.277945 seconds
NOTE:       cpu time                2.105680 seconds (757.59%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  69.16M (0.03%)
NOTE: Executing action 'astore.score'.
NOTE: Action 'astore.score' used (Total process time):
NOTE:       real time               0.141412 seconds
NOTE:       cpu time                0.584910 seconds (413.62%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  60.40M (0.02%)


[OutputCasTables]

             casLib                        Name   Rows  Columns  \
 0  CASUSER(bigebr)  preprocessed_curr_test_tbl  15200      150   
 
                                             casTable  
 0  CASTable('preprocessed_curr_test_tbl', caslib=...  

[Timing]

 Task Timing
 
                  Task   Seconds   Percent
 0   Loading the Store  0.000249  0.002107
 1  Creating the State  0.024531  0.207625
 2             Scoring  0.092737  0.784909
 3               Total  0.118150  1.000000

+ Elapsed: 0.141s, user: 0.465s, sys: 0.12s, mem: 60.4mb

In [127]:
"""
    Author: Biruk Gebremariam, PhD
    A one-hot expansion class using CAS data step.

"""
import swat
from enum import Enum

class BinaryType(Enum):
    ZERO_ONE = 1,
    MINUS_ONE_ONE = 2
    
class OneHotExpander:
    
    def __init__ (self, s, input_tbl, sparse_coded_cols):
        """
            s: the CAS session
            input_tbl: the CAS table to process
            sparse_coded_cols: the cols that are sparse/integer coded
        """
        self._s = s
        self._input_tbl = input_tbl
        self._sparse_coded_cols = [v.lower() for v in sparse_coded_cols]
        self._collect_stats()
    
    def expand(self, output_tbl, input_tbl=None, copy_vars=[],
               vars_to_expand=None, binary_type=BinaryType.ZERO_ONE,
               avoid_multicollinearity=False):
        """ if vars_to_expand is None, it defaults to _sparse_coded_cols
        """
        self._avoid_multicollinearity = avoid_multicollinearity
        if not input_tbl:
            input_tbl = self._input_tbl
        print(input_tbl, "here")
        if vars_to_expand:
            for var in vars_to_expand:
                var = var.lower()
                if var not in self._sparse_coded_cols:
                    sc = self._sparse_coded_cols
                    raise Exception("Invalid var: {0} - should be part of {1} ".format(var, sc))
        else:
            vars_to_expand = self._sparse_coded_cols
        code_str = self._generate_expansion_code(input_tbl, output_tbl, vars_to_expand, copy_vars,
                                                 binary_type=binary_type)
        self._s.dataStep.runCode(code=code_str)
        temp_vars = list(self._s.columnInfo(table=output_tbl)['ColumnInfo']['Column'])
        temp_vars = [v.lower() for v in temp_vars]
        temp_copied_vars = self._input_tbl_cols
        one_hot_vars = set(temp_vars) - set(temp_copied_vars)
        res = self._s.distinct(table=output_tbl, inputs=one_hot_vars, raw=True)['Distinct']
        vs = list(res['Column'])
        vs = [v.lower() for v in vs]
        n_levels_col = list(res['NDistinct'])
        n_levels_col = [int(lev) for lev in n_levels_col]
        return dict(zip(vs, n_levels_col))
        
    def _collect_stats(self):
        res = self._s.distinct(table=self._input_tbl, inputs=self._sparse_coded_cols, raw=True,
                               includeMissing=False)['Distinct']
        var_names_col = res['Column']
        var_names_col = [n.lower() for n in var_names_col]
        n_levels_col = res['NDistinct']
        n_levels_col = [int(lev) for lev in n_levels_col]
        self._n_levels_dict = dict(zip(var_names_col, n_levels_col))
        temp = list(self._s.columnInfo(table=self._input_tbl)['ColumnInfo']['Column'])
        self._input_tbl_cols = [v.lower() for v in temp]
    
    
    def _generate_expansion_code(self, input_tbl, output_tbl, vars_to_expand, copy_vars,
                                 binary_type=BinaryType.ZERO_ONE):
        """copy_vars:-the vars to copy in the output
        """
        def gen_one_hot_name(base_name, index):
            v = base_name + "_" + str(index)
            return v.lower()
        # setup the data step code.
        vars_to_expand = [v.lower() for v in vars_to_expand]
        all_one_hot_feat_names = []
        for var in vars_to_expand:
            n_levels = self._n_levels_dict[var.lower()]
            if self._avoid_multicollinearity:
                n_levels -= 1
            for i in range(n_levels):
                all_one_hot_feat_names.append(gen_one_hot_name(var, i+1))
        code_str = "data " + output_tbl 
        code_str += " (keep = "
        for v in all_one_hot_feat_names:
            code_str += v + " "
        for v in copy_vars:
            code_str += v + " "
        code_str += " )"
        code_str += ";"
        print(input_tbl)
        code_str += "set " + input_tbl + ";"
        for var in vars_to_expand:
            n_levels = self._n_levels_dict[var.lower()]
            if self._avoid_multicollinearity:
                n_levels -= 1
            for i in range(n_levels):
                index = i + 1
                one_hot_feat_name = var + "_" + str(index)
                code_str += "if " + var + " = " + str(index) + " then " + one_hot_feat_name +" = 1;"
                if binary_type == BinaryType.ZERO_ONE:
                    code_str += "else " + one_hot_feat_name + " = 0;"
                else:
                    code_str += "else " + one_hot_feat_name + " = -1;"
        code_str += "run;"
        return code_str



In [130]:
sparse_coded_cols = tr3_high_miss_intervals + tr4_low_miss_nominals + tr5_high_miss_nominals
copy_cols = tr1_low_miss_intervals + tr2_high_miss_intervals + [target]
            
ohe = OneHotExpander(s, preprocessed_curr_train_tbl, sparse_coded_cols)

NOTE: Executing action 'simple.distinct'.
NOTE: Action 'simple.distinct' used (Total process time):
NOTE:       real time               0.289786 seconds
NOTE:       cpu time                2.279654 seconds (786.67%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  108.13M (0.04%)
NOTE: Executing action 'table.columnInfo'.
NOTE: Action 'table.columnInfo' used (Total process time):
NOTE:       real time               0.002051 seconds
NOTE:       cpu time                0.001000 seconds (48.76%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  884.34K (0.00%)


In [131]:
ohe_preprocessed_curr_train_tbl = 'ohe_preprocessed_curr_train_tbl'
ohe.expand(ohe_preprocessed_curr_train_tbl, copy_vars=copy_cols, avoid_multicollinearity=True)


preprocessed_curr_train_tbl here
preprocessed_curr_train_tbl
NOTE: Executing action 'dataStep.runCode'.


NOTE: Duplicate messages output by DATA step:
NOTE: Action 'dataStep.runCode' used (Total process time):
NOTE:       real time               3087.068905 seconds
NOTE:       cpu time                92887.278988 seconds (3008.91%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  441.29M (0.17%)
NOTE: Executing action 'table.columnInfo'.
NOTE: Action 'table.columnInfo' used (Total process time):
NOTE:       real time               0.030216 seconds
NOTE:       cpu time                0.029995 seconds (99.27%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  6.81M (0.00%)


NOTE: Executing action 'simple.distinct'.
NOTE: Action 'simple.distinct' used (Total process time):
NOTE:       real time               21.977470 seconds
NOTE:       cpu time                498.749179 seconds (2269.37%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  7.61G (3.02%)


{'tr5_x93_559': 2,
 'tr5_x122_704': 2,
 'tr5_x36_452': 2,
 'tr5_x19_414': 2,
 'tr5_x6_284': 2,
 'tr5_x128_66': 2,
 'tr5_x19_332': 2,
 'tr5_x19_379': 2,
 'tr5_x122_541': 2,
 'tr5_x35_196': 2,
 'tr5_x33_434': 2,
 'tr5_x35_703': 2,
 'tr5_x122_447': 2,
 'tr5_x128_634': 2,
 'tr5_x34_194': 2,
 'tr5_x130_99': 2,
 'tr5_x93_479': 2,
 'tr5_x33_290': 2,
 'tr5_x19_7': 2,
 'tr5_x98_893': 2,
 'tr5_x52_114': 2,
 'tr5_x8_347': 2,
 'tr5_x128_413': 2,
 'tr5_x34_175': 2,
 'tr5_x122_615': 2,
 'tr5_x34_571': 2,
 'tr5_x36_842': 2,
 'tr5_x36_679': 2,
 'tr5_x34_743': 2,
 'tr5_x19_561': 2,
 'tr5_x128_708': 2,
 'tr5_x19_204': 2,
 'tr5_x35_518': 2,
 'tr5_x6_135': 2,
 'tr5_x128_249': 2,
 'tr5_x36_1020': 2,
 'tr5_x42_232': 2,
 'tr5_x92_150': 2,
 'tr5_x98_394': 2,
 'tr5_x8_20': 2,
 'tr5_x6_244': 2,
 'tr5_x8_216': 2,
 'tr5_x27_54': 2,
 'tr5_x19_795': 2,
 'tr5_x135_25': 2,
 'tr5_x33_372': 2,
 'tr5_x8_53': 2,
 'tr5_x35_785': 2,
 'tr5_x98_793': 2,
 'tr5_x34_580': 2,
 'tr5_x34_388': 2,
 'tr5_x93_472': 2,
 'tr5_x135_76':

In [132]:
ohe_preprocessed_curr_test_tbl = 'ohe_preprocessed_curr_test_tbl'
ohe.expand(ohe_preprocessed_curr_test_tbl, input_tbl=preprocessed_curr_train_tbl,
           copy_vars=copy_cols, avoid_multicollinearity=True)


preprocessed_curr_train_tbl here
preprocessed_curr_train_tbl
NOTE: Executing action 'dataStep.runCode'.


NOTE: Duplicate messages output by DATA step:
NOTE: Action 'dataStep.runCode' used (Total process time):
NOTE:       real time               3099.510320 seconds
NOTE:       cpu time                91991.468172 seconds (2967.94%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  78.19M (0.03%)
NOTE: Executing action 'table.columnInfo'.
NOTE: Action 'table.columnInfo' used (Total process time):
NOTE:       real time               0.031964 seconds
NOTE:       cpu time                0.031994 seconds (100.09%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  6.79M (0.00%)


NOTE: Executing action 'simple.distinct'.
NOTE: Action 'simple.distinct' used (Total process time):
NOTE:       real time               28.505458 seconds
NOTE:       cpu time                625.238950 seconds (2193.40%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  7.60G (3.01%)


{'tr5_x93_559': 2,
 'tr5_x122_704': 2,
 'tr5_x36_452': 2,
 'tr5_x19_414': 2,
 'tr5_x6_284': 2,
 'tr5_x128_66': 2,
 'tr5_x19_332': 2,
 'tr5_x19_379': 2,
 'tr5_x122_541': 2,
 'tr5_x35_196': 2,
 'tr5_x33_434': 2,
 'tr5_x35_703': 2,
 'tr5_x122_447': 2,
 'tr5_x128_634': 2,
 'tr5_x34_194': 2,
 'tr5_x130_99': 2,
 'tr5_x93_479': 2,
 'tr5_x33_290': 2,
 'tr5_x19_7': 2,
 'tr5_x98_893': 2,
 'tr5_x52_114': 2,
 'tr5_x8_347': 2,
 'tr5_x128_413': 2,
 'tr5_x34_175': 2,
 'tr5_x122_615': 2,
 'tr5_x34_571': 2,
 'tr5_x36_842': 2,
 'tr5_x36_679': 2,
 'tr5_x34_743': 2,
 'tr5_x19_561': 2,
 'tr5_x128_708': 2,
 'tr5_x19_204': 2,
 'tr5_x35_518': 2,
 'tr5_x6_135': 2,
 'tr5_x128_249': 2,
 'tr5_x36_1020': 2,
 'tr5_x42_232': 2,
 'tr5_x92_150': 2,
 'tr5_x98_394': 2,
 'tr5_x8_20': 2,
 'tr5_x6_244': 2,
 'tr5_x8_216': 2,
 'tr5_x27_54': 2,
 'tr5_x19_795': 2,
 'tr5_x135_25': 2,
 'tr5_x33_372': 2,
 'tr5_x8_53': 2,
 'tr5_x35_785': 2,
 'tr5_x98_793': 2,
 'tr5_x34_580': 2,
 'tr5_x34_388': 2,
 'tr5_x93_472': 2,
 'tr5_x135_76':

# 2. Interpretable and Highly Accurate Interaction Features

In [55]:
# compare the 3-variable interaction features with lightgbm based ...

n_feature_leaves = 5

picked_vars = ranked_intervals[:5]
#a hyperparameter:-
# for apsfailure data, n_preprocess_bins = 10 gives the best...
# of course, minNObsInBin is also another hyperparameter.
n_preprocess_bins = 4

triplets = list(combinations(ranked_intervals[:5], 3))
print("Number of triplet interactions: ", len(triplets))
pipelines = []
output_opts = dict(output=dict(noScoreTable=True, noScoreCode=True, noTable=True))
for i, triplet in enumerate(triplets):
    p1 = dict(name='tr'+str(3*i+1),
              inputs=triplet[0], 
              discretize=dict(method='quantile', args=dict(nbins=n_preprocess_bins, 
                                                           overrides=dict(binMissing=True))),
              **output_opts)
    p2 = dict(name='tr'+str(3*i+2),
              inputs=triplet[1], 
              discretize=dict(method='quantile', args=dict(nbins=n_preprocess_bins, 
                                                           overrides=dict(binMissing=True))),
              **output_opts)
    p3 = dict(name='tr'+str(3*i+3),
              inputs=triplet[2], 
              discretize=dict(method='quantile', args=dict(nbins=n_preprocess_bins, 
                                                           overrides=dict(binMissing=True))),
              **output_opts)
    p_inter = dict(name='tr_inter'+str(i+1),
                   events = event,
                   interaction = dict(
                       method = 'ordered',
                       synthesizer = 'nominal',
                       inputs = [p1['name'], p2['name'], p3['name']],
                       targets = 'tr_tar'),
                   cattrans = dict(method = 'dtree', 
                                   args=dict(treeCrit = 'entropy',
                                             minNBins=n_feature_leaves,
                                             maxNBins=n_feature_leaves,
                                             overrides=dict(minNObsInBin=5))))
    pipelines += [p1, p2, p3, p_inter]
tar_p = dict(name='tr_tar', inputs=target, cattrans=dict(method='label'),
             **output_opts)
pipelines.append(tar_p)

Number of triplet interactions:  10


In [56]:
pipelines

[{'name': 'tr1',
  'inputs': 'x100',
  'discretize': {'method': 'quantile',
   'args': {'nbins': 10, 'overrides': {'binMissing': True}}},
  'output': {'noScoreTable': True, 'noScoreCode': True, 'noTable': True}},
 {'name': 'tr2',
  'inputs': 'x9',
  'discretize': {'method': 'quantile',
   'args': {'nbins': 10, 'overrides': {'binMissing': True}}},
  'output': {'noScoreTable': True, 'noScoreCode': True, 'noTable': True}},
 {'name': 'tr3',
  'inputs': 'x18',
  'discretize': {'method': 'quantile',
   'args': {'nbins': 10, 'overrides': {'binMissing': True}}},
  'output': {'noScoreTable': True, 'noScoreCode': True, 'noTable': True}},
 {'name': 'tr_inter1',
  'events': '1',
  'interaction': {'method': 'ordered',
   'synthesizer': 'nominal',
   'inputs': ['tr1', 'tr2', 'tr3'],
   'targets': 'tr_tar'},
  'cattrans': {'method': 'dtree',
   'args': {'treeCrit': 'entropy',
    'minNBins': 5,
    'maxNBins': 5,
    'overrides': {'minNObsInBin': 5}}}},
 {'name': 'tr4',
  'inputs': 'x100',
  'discret

In [57]:
def get_inter_feat(inter_var_map, triplet):
    for k in inter_var_map:
        kvals = inter_var_map[k]
        found = True
        for v in triplet:
            if v in kvals[0] or v in kvals[1] or v in kvals[2]:
                continue
            else:
                found = False
                break
        if not found:
            continue
        else:
            return k, kvals
    return None, None




In [58]:

curr_astore = '_astore_model_'
res = s.dataPreprocess.transform(table=curr_train_tbl, pipelines=pipelines,
                                 savestate=dict(name=curr_astore, replace=True),
                                 **dpp_opts)
var_trans_info = res['VarTransInfo']
inter_var_map_df = var_trans_info.query("Input1 != ''")
inter_var_map = dict()
for i, row in inter_var_map_df.iterrows():
    inter_var_map[row['ResultVar']] = (row['Input1'], row['Input2'], row['Input3'])
print(inter_var_map)

NOTE: Executing action 'dataPreprocess.transform'.
NOTE: 27379 bytes were written to the table "_astore_model_" in the caslib "CASUSER(bigebr)".
NOTE: Action 'dataPreprocess.transform' used (Total process time):
NOTE:       real time               0.271074 seconds
NOTE:       cpu time                3.283500 seconds (1211.29%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  28.16M (0.01%)
{'tr_inter1_var_1': ('tr1_x100', 'tr2_x9', 'tr3_x18'), 'tr_inter10_var_1': ('tr28_x18', 'tr29_x101', 'tr30_x10'), 'tr_inter2_var_1': ('tr4_x100', 'tr5_x9', 'tr6_x101'), 'tr_inter3_var_1': ('tr7_x100', 'tr8_x9', 'tr9_x10'), 'tr_inter4_var_1': ('tr10_x100', 'tr11_x18', 'tr12_x101'), 'tr_inter5_var_1': ('tr13_x100', 'tr14_x18', 'tr15_x10'), 'tr_inter6_var_1': ('tr16_x100', 'tr17_x101', 'tr18_x10'), 'tr_inter7_var_1': ('tr19_x9', 'tr20_x18', 'tr21_x101'), 'tr_inter8_var_1': ('tr22_x9', 'tr23_x18', 'tr24_x10'), 'tr_inter9_var_

In [59]:
curr_train_tbl_scored = curr_train_tbl + "_scored_"
s.astore.score(table=curr_train_tbl,
               rstore=curr_astore,
               copyvars = target,
               casout=dict(name=curr_train_tbl_scored, replace=True))
curr_train_tbl_scored_df = s.fetch(table=curr_train_tbl_scored, **fetch_opts)['Fetch']

NOTE: Executing action 'astore.score'.
NOTE: Action 'astore.score' used (Total process time):
NOTE:       real time               0.034478 seconds
NOTE:       cpu time                0.322951 seconds (936.69%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  45.42M (0.02%)
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               0.139734 seconds
NOTE:       cpu time                0.138978 seconds (99.46%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  23.17M (0.01%)


In [60]:
curr_test_tbl_scored = curr_test_tbl + "_scored_"
s.astore.score(table=curr_test_tbl,
               rstore=curr_astore,
               copyvars = target,
               casout=dict(name=curr_test_tbl_scored, replace=True))
curr_test_tbl_scored_df = s.fetch(table=curr_test_tbl_scored, **fetch_opts)['Fetch']

NOTE: Executing action 'astore.score'.
NOTE: Action 'astore.score' used (Total process time):
NOTE:       real time               0.026098 seconds
NOTE:       cpu time                0.168974 seconds (647.46%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  41.27M (0.02%)
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               0.066654 seconds
NOTE:       cpu time                0.066990 seconds (100.50%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  6.47M (0.00%)


In [61]:
inter_feats = list(inter_var_map.keys())
fte_triplets_score_info = dict()
prior = np.mean(curr_train_tbl_scored_df[target])
for inter_feat in inter_feats:
    means_df = curr_train_tbl_scored_df.groupby(inter_feat)[target].mean()
    if 0 not in means_df:
        means_df.at[0] = prior
    temp_key = []
    for k in inter_var_map[inter_feat]:
        temp_key.append(k[k.index("_")+1:])
    temp_key = tuple(temp_key)
    fte_triplets_score_info[temp_key] = dict()
    for i, temp_tbl in enumerate([curr_train_tbl_scored_df, curr_test_tbl_scored_df]):     
        act_vals = temp_tbl[target]
        pred_vals = np.empty(len(temp_tbl))
        for j in range(len(act_vals)):
            grp = temp_tbl[inter_feat][j]
            if not np.isnan(grp):
                pred_vals[j] = means_df[grp]
            else:
                pred_vals[j] = prior
        auc = roc_auc_score(act_vals, pred_vals)
        if i == 0:
            fte_triplets_score_info[temp_key]['train_auc'] = auc
        else:
            fte_triplets_score_info[temp_key]['test_auc'] = auc

        

In [62]:
fte_triplets_score_info.keys()

dict_keys([('x100', 'x9', 'x18'), ('x18', 'x101', 'x10'), ('x100', 'x9', 'x101'), ('x100', 'x9', 'x10'), ('x100', 'x18', 'x101'), ('x100', 'x18', 'x10'), ('x100', 'x101', 'x10'), ('x9', 'x18', 'x101'), ('x9', 'x18', 'x10'), ('x9', 'x101', 'x10')])

In [63]:
fte_triplets_score_info

{('x100', 'x9', 'x18'): {'train_auc': 0.5169975100643557,
  'test_auc': 0.5262296823102659},
 ('x18', 'x101', 'x10'): {'train_auc': 0.8226636915592972,
  'test_auc': 0.7948822640610338},
 ('x100', 'x9', 'x101'): {'train_auc': 0.7511201917914636,
  'test_auc': 0.7202161190515384},
 ('x100', 'x9', 'x10'): {'train_auc': 0.8142608193474882,
  'test_auc': 0.785413488679669},
 ('x100', 'x18', 'x101'): {'train_auc': 0.7558095470673625,
  'test_auc': 0.7309263282424383},
 ('x100', 'x18', 'x10'): {'train_auc': 0.8167421072967569,
  'test_auc': 0.792755236232915},
 ('x100', 'x101', 'x10'): {'train_auc': 0.8202918810274135,
  'test_auc': 0.7917378496991643},
 ('x9', 'x18', 'x101'): {'train_auc': 0.7556726252288336,
  'test_auc': 0.7340195760984195},
 ('x9', 'x18', 'x10'): {'train_auc': 0.8166796517212876,
  'test_auc': 0.7927149232282757},
 ('x9', 'x101', 'x10'): {'train_auc': 0.8202918810274135,
  'test_auc': 0.7917378496991643}}

In [64]:
import numpy as np
import pandas as pd
import swat 
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

class LabelEncoderWithUnseenSupport:
    """ maps unseen/missing etc to 0. The others are mapped from 1, 2,,..
    """
    def __init__(self):
        self._le = LabelEncoder()
        self._le_dict = None

    def fit(self, col):
        # nan in col creates troubles during the _le_dict creation 
        # due to le.transform failure... well that is the whole point 
        # for the creation of this class
        col = pd.Series(col).dropna().values
        self._le.fit(col)
        self._le_dict = dict(zip(self._le.classes_, 
                                 np.array(self._le.transform(self._le.classes_)) + 1))
    
    def transform(self, col):
        return pd.Series(col).apply(lambda x: self._le_dict.get(x, 0))
    
class LabelEncoderWrapper:

    def __init__(self, nominals=None):
        if nominals:
            self._nominals = [v.lower() for v in nominals]
            self._les = [LabelEncoderWithUnseenSupport() for i in range(len(nominals))]
        else:
            self._nominals = []
            self._les = []
        self._name_case_map = dict()
    
    def fit(self, train_df):
        cols = train_df.columns
        if not self._nominals:
            self._init_basics(train_df)
        for c in cols:
            self._name_case_map[c.lower()] = c
        for c in cols:
            try:
                i = self._nominals.index(c.lower())
                self._les[i].fit(train_df[c])
            except:
                # not in nominals... simply skip. will return all -1.
                pass
                
    
    def transform(self, input_df, copy=True, set_as_category=False, excludes=[]):
        df = pd.DataFrame()
        cols = input_df.columns
        for c in cols:
            if c in excludes or c.lower() in excludes:
                continue
            try:
                i = self._nominals.index(c.lower())
                vals = self._les[i].transform(input_df[c])
                df[c] = vals
                if set_as_category:
                    df[c] = df[c].astype('category')
            except:
                # not in nominals... simply continue
                if copy:
                    df[c] = input_df[c]
        return df


    def _init_basics(self, train_df):
        temp_df = train_df.dtypes
        temp_cols = temp_df.index
        temp_types = temp_df.values
        for i in range(len(temp_cols)):
            t = str(temp_types[i])
            if not ('int' in t or 'float' in t):
                self._nominals.append(temp_cols[i].lower())           
                self._les.append(LabelEncoderWithUnseenSupport())
                
class DTreeLGB:
    
    TRANSFORM_OPTS = dict(misraGries=False, distinctCountLimit=10000000)
    
    FETCH_OPTS = dict(to=10000000, maxrows=1000000)
    
    def __init__(self, session):
        self._session = session
        if self._session:
            self._session.loadactionset(actionset='dataPreprocess')
            self._session.loadactionset(actionset='astore')
        self._estimator = None
    
    def train(self, train_df, target, inputs, nominals=[], n_leaves=32, max_bin=100, 
              use_transform=True):
        self._target = target
        self._inputs = list(set(inputs) - set([target]))
        self._nominals = list(set(nominals) - set([target]))
        self._intervals = list((set(inputs) - set(nominals)) - set([target]))
        if nominals:
            train_df = train_df[self._inputs + [self._target]]
            if use_transform:                
                temp_input = '__temp_input__'
                self._session.upload(train_df, casout=dict(name=temp_input, replace=True))
                self._prepare_astore_for_data_trans(temp_input)
                train_df = self._prepare_data(temp_input)
                train_df.to_csv("U:\\to_csv1.csv", index=False)
            else:
                self._data_prep_astore = None
                self._le = LabelEncoderWrapper(nominals=self._nominals)
                self._le.fit(train_df)
                train_df = self._le.transform(train_df, copy=True, set_as_category=True)
                train_df.to_csv("U:\\to_csv2.csv", index=False)
        train_df = train_df[self._intervals + self._nominals + [self._target]]
        np.random.seed(1)
        y_train = train_df[self._target]
        x_train = train_df[self._intervals + self._nominals]
        lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  
                                           objective='binary', 
                                           # binary_logloss - aliased with binary
                                           # auc, binary_error etc possible
                                           metric = ['logloss'],
                                           num_leaves = n_leaves,
                                           max_depth=-1, 
                                           # note that leraning rate has to be set to 1
                                           # so that we have a single decision tree ...
                                           # note that gbt is :- y_pred = mean_y + learn_rate*(y_curr - mean_y) + ....
                                           # so setting learn_rate to 1 makes y_pred = y_curr which is what we want
                                           # where y_curr is the prediction from the current/single tree
                                           learning_rate=1, 
                                           n_estimators=1, 
                                           subsample_for_bin=1000000, 
                                           class_weight=None, 
                                           min_split_gain=0.0, 
                                           min_child_weight=0.001, 
                                           min_child_samples=5, 
                                           subsample=1.0, 
                                           subsample_freq=0, 
                                           colsample_bytree=1.0, 
                                           reg_alpha=0.0, 
                                           reg_lambda=0.0, 
                                           random_state=None, 
                                           n_jobs=-1, 
                                           silent=False, 
                                           importance_type='split',
                                           max_bin = max_bin)
        self._estimator = lgb_estimator
        self._estimator.fit(x_train, y_train)
    
    def predict(self, df):
        if self._nominals:
            df = df[self._inputs + [self._target]]
            if self._data_prep_astore:
                temp_input = '__temp_input__'
                self._session.upload(df, casout=dict(name=temp_input, replace=True))
                df = self._prepare_data(temp_input)
            else:
                df = self._le.transform(df, copy=True, set_as_category=True)                
        df = df[self._intervals + self._nominals + [self._target]]
        x_df = df[self._intervals + self._nominals]
        y_pred = self._estimator.predict_proba(x_df, raw_score=False)
        scored_df = df.copy()
        for i, c in enumerate(self._estimator.classes_):
            try:
                c = int(c)
            except:
                pass
            scored_df[self._target + "_" + str(c)+"_pred"] = y_pred[:,i]
        return scored_df

    def _prepare_data(self, input_tbl_name):
        temp_output = '__temp_output__'
        self._session.astore.score(table=input_tbl_name,
                                   rstore=self._data_prep_astore,
                                   casout=dict(name=temp_output, replace=True),
                                   copyvars = [self._target] + self._intervals)
        temp_res = self._session.fetch(table=temp_output, **DTreeLGB.FETCH_OPTS)
        df = temp_res['Fetch']
        cols = list(df.columns)
        self._nom_indices = []
        for i, c in enumerate(cols):
            if c in self._nominals:
                self._nom_indices.append(i)
                df[c] = df[c].astype('category')
            else:
                df[c] = df[c]
        return df

    def _prepare_astore_for_data_trans(self, input_tbl_name):
        astore_name = "_lgb_data_prep_astore_"
        pipelines = []
        p1 = dict(name='tr1',
                  inputs = self._nominals,
                  cattrans = dict(method = 'label',
                                  args = dict(overrides = dict(binMissing=True))))
        pipelines = [p1]
        self._session.dataPreprocess.transform(table=input_tbl_name,
                                               pipelines=pipelines,
                                               saveState = dict(name=astore_name, replace=True),
                                               casout = dict(name='__ttt___', replace=True),
                                               **DTreeLGB.TRANSFORM_OPTS)
        #because of a bug in astore (communicated to YQ, can't use 
        # outVarsNameGlobalPrefix='' (when mapping non-numeric to numeric 
        # using the same input - output name))...hance map the nominals names
        nominals = []
        for n in self._nominals:
            nominals.append(p1['name']+"_" + n)
        self._nominals = nominals
        self._data_prep_astore = astore_name
        return
    



In [65]:
triplets

[('x100', 'x9', 'x18'),
 ('x100', 'x9', 'x101'),
 ('x100', 'x9', 'x10'),
 ('x100', 'x18', 'x101'),
 ('x100', 'x18', 'x10'),
 ('x100', 'x101', 'x10'),
 ('x9', 'x18', 'x101'),
 ('x9', 'x18', 'x10'),
 ('x9', 'x101', 'x10'),
 ('x18', 'x101', 'x10')]

In [66]:
curr_train_tbl_fetched = s.fetch(table=curr_train_tbl, **fetch_opts)['Fetch']
curr_test_tbl_fetched = s.fetch(table=curr_test_tbl, **fetch_opts)['Fetch']

lgb_triplets_score_info = dict()
for triplet in triplets:
    lgb_dtree = DTreeLGB(None)

    lgb_inputs = triplet
    lgb_nominals = []
    lgb_dtree.train(curr_train_tbl_fetched, target, lgb_inputs, nominals=lgb_nominals, 
                n_leaves=n_feature_leaves, max_bin=256,
                use_transform=False)
    curr_train_tbl_fetched_scored = lgb_dtree.predict(curr_train_tbl_fetched)
    curr_test_tbl_fetched_scored = lgb_dtree.predict(curr_test_tbl_fetched)
    y_train_true = curr_train_tbl_fetched_scored[target]
    y_train_scored = curr_train_tbl_fetched_scored[target+"_1_pred"]
    auc_train = roc_auc_score(y_train_true, y_train_scored)
    y_test_true = curr_test_tbl_fetched_scored[target]
    y_test_scored = curr_test_tbl_fetched_scored[target+"_1_pred"]
    auc_test = roc_auc_score(y_test_true, y_test_scored)
    print("auc train: ", auc_train)
    print("auc test: ", auc_test)
    
    lgb_triplets_score_info[triplet] = dict(train_auc=auc_train, test_auc=auc_test)

NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               5.780470 seconds
NOTE:       cpu time                5.588151 seconds (96.67%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  209.30M (0.08%)
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               1.862862 seconds
NOTE:       cpu time                1.820723 seconds (97.74%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            252.23G
NOTE:       memory                  53.25M (0.02%)
auc train:  0.8396212811007747
auc test:  0.831717367591807
auc train:  0.7326756641129543
auc test:  0.7085763850036657
auc train:  0.8224869542914066
auc test:  0.7894902704821719
auc train:  0.8178044150807153
auc test:  0.821246969095172
auc train:  0.7989966466760783
auc test:  0.79885567918

In [67]:
lgb_triplets_score_info

{('x100', 'x9', 'x18'): {'train_auc': 0.8396212811007747,
  'test_auc': 0.831717367591807},
 ('x100', 'x9', 'x101'): {'train_auc': 0.7326756641129543,
  'test_auc': 0.7085763850036657},
 ('x100', 'x9', 'x10'): {'train_auc': 0.8224869542914066,
  'test_auc': 0.7894902704821719},
 ('x100', 'x18', 'x101'): {'train_auc': 0.8178044150807153,
  'test_auc': 0.821246969095172},
 ('x100', 'x18', 'x10'): {'train_auc': 0.7989966466760783,
  'test_auc': 0.7988556791849744},
 ('x100', 'x101', 'x10'): {'train_auc': 0.7070924941850256,
  'test_auc': 0.682752287698409},
 ('x9', 'x18', 'x101'): {'train_auc': 0.8042738050026436,
  'test_auc': 0.8082005437604126},
 ('x9', 'x18', 'x10'): {'train_auc': 0.8046749244347591,
  'test_auc': 0.8086510932705967},
 ('x9', 'x101', 'x10'): {'train_auc': 0.8224869542914066,
  'test_auc': 0.7894902704821719},
 ('x18', 'x101', 'x10'): {'train_auc': 0.7988815993023232,
  'test_auc': 0.7985804654033019}}

In [68]:
fte_triplets_score_info

{('x100', 'x9', 'x18'): {'train_auc': 0.5169975100643557,
  'test_auc': 0.5262296823102659},
 ('x18', 'x101', 'x10'): {'train_auc': 0.8226636915592972,
  'test_auc': 0.7948822640610338},
 ('x100', 'x9', 'x101'): {'train_auc': 0.7511201917914636,
  'test_auc': 0.7202161190515384},
 ('x100', 'x9', 'x10'): {'train_auc': 0.8142608193474882,
  'test_auc': 0.785413488679669},
 ('x100', 'x18', 'x101'): {'train_auc': 0.7558095470673625,
  'test_auc': 0.7309263282424383},
 ('x100', 'x18', 'x10'): {'train_auc': 0.8167421072967569,
  'test_auc': 0.792755236232915},
 ('x100', 'x101', 'x10'): {'train_auc': 0.8202918810274135,
  'test_auc': 0.7917378496991643},
 ('x9', 'x18', 'x101'): {'train_auc': 0.7556726252288336,
  'test_auc': 0.7340195760984195},
 ('x9', 'x18', 'x10'): {'train_auc': 0.8166796517212876,
  'test_auc': 0.7927149232282757},
 ('x9', 'x101', 'x10'): {'train_auc': 0.8202918810274135,
  'test_auc': 0.7917378496991643}}

In [69]:

v1 = []
v2 = []
v3 = []
lgb_test_aucs = []
fte_test_aucs = []
for triplet in triplets:
    v1.append(triplet[0])
    v2.append(triplet[1])
    v3.append(triplet[2])
    lgb_test_aucs.append(lgb_triplets_score_info[triplet]['test_auc'])
    fte_test_aucs.append(fte_triplets_score_info[triplet]['test_auc'])
temp_df = pd.DataFrame()
temp_df['v1'] = v1
temp_df['v2'] = v2
temp_df['v3'] = v3
temp_df['lgb_test_auc'] = lgb_test_aucs
temp_df['fte_test_auc'] = fte_test_aucs
temp_df.to_csv(str(Path(data_path) / 'output' / str(tbl_base_name + '_triplets.csv')), index=False)

In [70]:
s.close()

In [ ]:
impute_phase = dict(method='median')
disct_phase = dict(
    method = 'quantile', args = dict(nBins=5)
)
p1 = dict(name='tr1',
          inputs=['x1', 'x2', 'x3'],
          impute = impute_phase,
          discretize = disct_phase)
p2 = p1; p2['inputs'] = ['x11', 'x12', 'x15']
p3 = p1; p3['inputs'] = ['x111', 'x112', 'x5']
cattrans_phase = dict(method = 'dtree', 
                      args=dict(treeCrit = 'entropy',
                                nBins = 5))
inter_inputs = [p1['name'], p2['name'], p3['name']]
p_inter = dict(name='tr_inter',
               interaction = dict(
                   synthesizer = 'nominal',
                   inputs = inter_inputs,
                   targets = p_tar['name']),
               cattrans = cattrans_phase)